# Workshop DL03: Recurrent Neural Networks

## Workshop Exercises
* **Exercise 1**. Find your own text corpus to build an RNN on.  
* **Exercise 2**. Implement your own RNN using your text corpus as training data. Feel free to use the `chunkify/encode` functions provided below to help with preprocessing. If you get stuck on anything, then follow the MNIST worked example provided.

## Recurrent Neural Networks
* http://adventuresinmachinelearning.com/recurrent-neural-networks-lstm-tutorial-tensorflow/
* http://karpathy.github.io/2015/05/21/rnn-effectiveness/

## Text Preprocessing

In [295]:
text = "long ago , the mice had a general council to consider what measures they could take to outwit their common enemy , the cat . some said this , and some said that but at last a young mouse got up and said he had a proposal to make , which he thought would meet the case . you will all agree , said he , that our chief danger consists in the sly and treacherous manner in which the enemy approaches us . now , if we could receive some signal of her approach , we could easily escape from her . i venture , therefore , to propose that a small bell be procured , and attached by a ribbon round the neck of the cat . by this means we should always know when she was about , and could easily retire while she was in the neighbourhood . this proposal met with general applause , until an old mouse got up and said that is all very well , but who is to bell the cat ? the mice looked at one another and nobody spoke . then the old mouse said it is easy to propose impossible remedies ."

In [296]:
# split a given passage of text into chunks of the specified size
def chunkify(text, chunk_size):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i:i+chunk_size])
    return chunks

In [297]:
# converts a string into a list of integers
def encode_string(string):
    encoding = []
    for char in string:
        encoding.append([float(ord(char))])
    return encoding

# converts a list of strings into a list of lists of integers
def encode_text(text):
    encodings = []
    for string in text:
        encodings.append(encode_string(string))
    while len(encodings[-1]) < len(encodings[0]):
        encodings[-1].append([float(ord(" "))])
    return encodings

In [298]:
def preprocess(text, sequence_length):
    train_X = encode_text(chunkify(text, sequence_length))
    train_y = [train_X[i+1][0] for i in range(0, len(train_X)-1)]
    train_y.append(float(ord(" ")))
    return train_X, train_y

## Worked Example: MNIST RNN

In [86]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import rnn

In [87]:
mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [88]:
N_CLASSES = 10

N_SEQUENCES = 28
SEQUENCE_SIZE = 28
RNN_SIZE = 128

N_TRAINING_EPOCHS = 1
TRAINING_BATCH_SIZE = 128

In [101]:
tf.reset_default_graph()

In [102]:
with tf.variable_scope("input") as scope:
    X = tf.placeholder(tf.float32, shape=[None, 784])
    X_box = tf.reshape(X, [-1, N_SEQUENCES, SEQUENCE_SIZE])
    X_transpose = tf.transpose(X_box, [1,0,2])
    X_reshape = tf.reshape(X_transpose, [-1, SEQUENCE_SIZE])
    X_split = tf.split(X_reshape, N_SEQUENCES, 0)
    
with tf.variable_scope("rnn") as scope:
    lstm_cell = rnn.BasicLSTMCell(RNN_SIZE)
    outputs, states = rnn.static_rnn(lstm_cell, X_split, dtype=tf.float32)
    
with tf.variable_scope("output") as scope:
    W = tf.get_variable("weights", shape=[RNN_SIZE, N_CLASSES])
    b = tf.get_variable("biases", shape=[N_CLASSES])
    y_ = tf.matmul(outputs[-1], W) + b

In [103]:
with tf.variable_scope("loss") as scope:
    y = tf.placeholder(tf.float32, shape=[None, N_CLASSES])
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        labels=y, logits=y_))

with tf.variable_scope("optimizer") as scope:
    optimizer = tf.train.AdamOptimizer().minimize(loss)

In [105]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(N_TRAINING_EPOCHS):
        epoch_loss = 0
        
        for i in range(int(mnist.train.num_examples/TRAINING_BATCH_SIZE)):
            epoch_x, epoch_y = mnist.train.next_batch(TRAINING_BATCH_SIZE)
            
            _, c = sess.run([optimizer, loss], feed_dict={
                X: epoch_x,
                y: epoch_y
            })
            epoch_loss += c
            
        print("Epoch " + str(epoch) + " completed.")
        print("Loss: " + str(epoch_loss))

Epoch 0 completed.
Loss: 234.14081805944443


## Ignore This - Worked Example: Character Array

In [335]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [351]:
BATCH_LEN = 24
SEQUENCE_LEN = 8
ELEMENT_LEN = 1
N_CLASSES = 1
N_TRAINING_EPOCHS = 10

In [352]:
train_X, train_y = preprocess(text, SEQUENCE_LEN)
batches_X, batches_y = chunkify(train_X, BATCH_LEN), chunkify(train_y, BATCH_LEN)

In [353]:
tf.reset_default_graph()

In [354]:
with tf.variable_scope("input"):
    X = tf.placeholder(tf.float32,
        shape=[BATCH_LEN, SEQUENCE_LEN, ELEMENT_LEN])
    X_reshape = tf.reshape(X, [-1, SEQUENCE_LEN])
    X_split = tf.split(X_reshape, SEQUENCE_LEN, 1)

In [355]:
with tf.variable_scope("rnn"):
    lstm_cell = rnn.MultiRNNCell(
        [rnn.BasicLSTMCell(128),
        rnn.BasicLSTMCell(128),
        rnn.BasicLSTMCell(128)]
    )
    output, state = tf.nn.static_rnn(
        lstm_cell, X_split, dtype=tf.float32)

In [356]:
with tf.variable_scope("output"):
    W = tf.get_variable("weight", shape=[128, 1])
    b = tf.get_variable("biases", shape=[1])
    y_ = tf.matmul(output[-1], W) + b

In [357]:
with tf.variable_scope("loss"):
    y = tf.placeholder(tf.float32, shape=[BATCH_LEN, N_CLASSES])
    msqerr = (y - y_)**2
    loss = tf.reduce_mean(msqerr)

In [358]:
with tf.variable_scope("optimizer"):
    optimizer = tf.train.RMSPropOptimizer(0.01).minimize(loss)

In [360]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(1, N_TRAINING_EPOCHS+1):
        for i in range(0, len(batches_X)-1):
            sess.run([optimizer], feed_dict={X:batches_X[i], y:batches_y[i]})